In [1]:
import pickle,tensorflow
GloVe_Corpus_Index=pickle.load(open('glove300.pickle','rb'))
Corpus_Token_Index=pickle.load(open('Corpus_Token.pickle','rb'))
LSTM_Units,Longest_Sequence,Target_Label_Weights = pickle.load(open('Sentence_Restoration_Variables.pickle','rb'))

def Weighted_Loss(Target_Label_Weights):
    def innerLoss(Actual, Predicted):
        import tensorflow
        return tensorflow.reshape(tensorflow.gather(Target_Label_Weights, Actual), (-1,Longest_Sequence)) \
               * \
         tensorflow.keras.losses.SparseCategoricalCrossentropy(reduction=tensorflow.keras.losses.Reduction.NONE)(Actual,Predicted)
    return innerLoss

Model=tensorflow.keras.models.load_model('Restore_Sentence_'+str(LSTM_Units)+'_unit_Triple_BiLSTM_'+str(Longest_Sequence)+'MaxToken.keras',
                                        custom_objects={ 'innerLoss': Weighted_Loss(Target_Label_Weights) })

In [2]:
def Get_YouTube_Transcripts():
    import pyodbc
    DB = pyodbc.connect(r'Driver={SQL Server};Server=(local);Database=YouTube;Trusted_Connection=yes;',autocommit=True)
    DB_Link=DB.cursor()
    Transcripts_Rows=DB_Link.execute("Select Captions \
                                        From Video \
                                        Where Captions Is Not Null").fetchall()
    DB_Link.close()
    DB.close()
    Transcripts=[Row[0] for Row in Transcripts_Rows]
    return(Transcripts)

Transcripts = Get_YouTube_Transcripts()

In [3]:
import numpy, nltk
Tokenizer = nltk.tokenize.treebank.TreebankWordTokenizer()
DeTokenizer = nltk.tokenize.treebank.TreebankWordDetokenizer()
i=0
for Transcript in Transcripts:
    if i==0:
        i+=1
        continue # because first one in my list is uninsteresting 
    Tokens_Integers=[Corpus_Token_Index[Token] if Token in Corpus_Token_Index else 1 for Token in Tokenizer.tokenize(Transcript)]
    Tokens_Integers=Tokens_Integers+[0]*((int(Longest_Sequence/4))-len(Tokens_Integers)%(int(Longest_Sequence/4))) # padding so that all predictions made on Longest_Sequence-sized subsets
    Sentence_Location=numpy.zeros(len(Tokens_Integers),dtype=numpy.float64)
    for Subset_Number in range(int((len(Tokens_Integers)//(Longest_Sequence/4))-3)):
        for (Token_Location,Label) in [(int(Subset_Location+Subset_Number*(Longest_Sequence/4)), 1.0 if Predicted_Label==2 else 0.0) for Subset_Location,Predicted_Label in enumerate(numpy.argmax(Model.predict(verbose=0,x=numpy.array([Tokens_Integers[int(Subset_Number*(Longest_Sequence/4)):int(Subset_Number*(Longest_Sequence/4)+(Longest_Sequence))]])), axis=-1)[0,])]:
            Sentence_Location[Token_Location]+=Label/(4 if Token_Location >=(3*Longest_Sequence/4) and Token_Location < len(Tokens_Integers)-192 else 3 if Token_Location>=(Longest_Sequence/2) and Token_Location < len(Tokens_Integers)-128 else 2 if Token_Location>=(Longest_Sequence/4) and Token_Location < len(Tokens_Integers)-64 else 1)
    Sentence=[];Sentences=[];Next_Token_Starts_Sentence=True
    for Token_Location,Token in enumerate(Tokenizer.tokenize(Transcript)):
        if round(Sentence_Location[Token_Location],0)==1.0:
            if len(Sentence)==0 and Next_Token_Starts_Sentence:
                Sentences.append(Token.capitalize()+'.')
            else:
                Sentence.append(Token+'.')
                Sentences.append(DeTokenizer.detokenize(Sentence))
            Next_Token_Starts_Sentence=True
            Sentence=[]
        else:
            if Next_Token_Starts_Sentence:
                Sentence.append(Token.capitalize())
                Next_Token_Starts_Sentence=False
            else:
                Sentence.append(Token)
    print(' '.join(Sentences))
    print('')
    break #just do one transcript as an example

Hey folks. Dr mike ezratal here for renaissance periodization nucleus overload training. You've been asking about it. And i'm answering. We're going to take a critical look upsides. Downsides so on and so forth. Now before we start different folks have different ideas slightly of what this means. We're sort of shooting for the average. So i'm really sorry. If i mischaracterize it. I promise it wasn't on purpose. I did some interneting to try to find out what it is. That it really was. I found out some stuff. Hopefully. I'm not too far off the mark. So today we're going to talk about what nucleus overload training is. What are the potential benefits to it. What are some unanswered questions about it. And what are the potential. Uh. Upsides. Or sorry. Potential downsides. Or those benefits unanswered. Questions downsides. And then lastly should you try it because you're probably thinking should i try. Let's get started. So first of all what is nucleus overload training. It's essentially.